# instalación de paquetes y verificación

In [21]:
# comprobamos que la versión de python sea inferior a 3.8, ya que para usar tensorflow 1.15 se recomienda usar python
# 3.7 o inferior
import sys
import string
python_version=sys.version.split("(")[0]
if int(sys.version.split(".")[1])>7:
    print("you are using a python version higer than 3.7.x, please install python 3.7.x")
else:
    print("congrats, you are using {} which is compatible with this notebook".format(python_version))


congrats, you are using 3.7.9  which is compatible with this notebook


In [2]:
# vamos a instalar todos los paquetes necesarios para el funcionamiento de este cuaderno. Este cuaderno, junto con el
# repo de github con las carpetas y archivos nos ahorran tener que seguir el proceso descrito aqu: 
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1.md
# y además nos evitan posibles problemas
!pip install -r requirements.txt

In [3]:
# si todo sale bien,el test debería correr y los resultados aparecerán debajo. Es normal si se salta alguno de los tests 
# que hay
!python3 object_detection/builders/model_builder_tf1_test.py


Running tests under Python 3.7.9: /home/bigdata/anaconda3/envs/test/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feat

# creación del dataset

In [1]:
!tree -d images

images
├── test
│   ├── JPEGImages
│   └── Visualization
├── train
│   ├── JPEGImages
│   └── Visualization
└── val
    ├── JPEGImages
    └── Visualization

9 directories


In [ ]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test
!python3 scripts/train_test_split.py -i images -o images -tr 0.1 -vr 0.15
!tree -d images

In [ ]:
# vamos a convertir estas fotos en y anotaciones en datasets de formato COCO. Tenemos que hacerlo para los sets de train
# test y validación. Tenemos que pasar como argumento archivo txt con los labels siguiendo el formato indiado en la
# documentación oficial de labelme https://github.com/wkentaro/labelme/tree/master/examples/instance_segmentation. Un ejemplo
# de cómo debe ser este archivo se muestra debajo
!python3 scripts/labelme2coco.py images/train images/train/coco --labels training/labels.txt
!python3 scripts/labelme2coco.py images/test images/test/coco --labels training/labels.txt
!python3 scripts/labelme2coco.py images/val images/val/coco --labels training/labels.txt
!tree -d images

<img src="training/ejemplo_labels_txt.PNG">

In [ ]:
# ahora borramos las imágenes y anotaciones de las carpetas train, test y val, ya que la carpeta coco dentro de cada 
# una de estas carpetas ya contiene las imágenes originales
!rm images/test/*
!mv images/test/coco/* images/test
!rm -rf images/test/coco 

!rm images/train/*
!mv images/train/coco/* images/train
!rm -rf images/train/coco 

!rm images/val/*
!mv images/val/coco/* images/val
!rm -rf images/val/coco 

!tree -d images

In [3]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir=images/train \
--val_image_dir=images/val \
--test_image_dir=images/test \
--train_annotations_file=images/train/annotations.json \
--val_annotations_file=images/val/annotations.json \
--testdev_annotations_file=images/test/annotations.json \
--output_dir=training/tfrecords \
--include_masks=True

I0902 08:29:11.932793 140572526618432 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I0902 08:29:11.933215 140572526618432 create_coco_tf_record.py:412] 0 images are missing annotations.
I0902 08:29:11.933476 140572526618432 create_coco_tf_record.py:441] On image 0 of 117
I0902 08:29:35.454863 140572526618432 create_coco_tf_record.py:441] On image 100 of 117
I0902 08:29:39.719069 140572526618432 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I0902 08:29:39.724019 140572526618432 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I0902 08:29:39.724163 140572526618432 create_coco_tf_record.py:412] 0 images are missing annotations.
I0902 08:29:39.724201 140572526618432 create_coco_tf_record.py:441] On image 0 of 24
I0902 08:29:45.580536 140572526618432 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I0902 08:29:45.583121 140572526618432 create_coco_tf_record.py:39

# Entrenamiento y exportación del modelo

In [2]:
# entrenamos el modelo usando estos tfrecords. El modelo resultante se guardará el carpeta training
!python3 object_detection/model_main.py --alsologtostderr \
--model_dir=training/ \
--pipeline_config_path=training/mask_rcnn_inception_v2_coco.config \
--num_train_steps=400000

W0902 11:11:03.384442 140297466345280 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 400000
I0902 11:11:03.384585 140297466345280 config_util.py:552] Maybe overwriting train_steps: 400000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0902 11:11:03.384632 140297466345280 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0902 11:11:03.384671 140297466345280 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0902 11:11:03.384709 140297466345280 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0902 11:11:03.384760 140297466345280 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_

Instructions for updating:
Use `tf.cast` instead.
W0902 11:11:15.061349 140297466345280 deprecation.py:323] From /home/bigdata/Documentos/proyecto_mario/tensorflow_object_detection_guide/object_detection/inputs.py:259: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
I0902 11:11:17.867290 140297466345280 estimator.py:1148] Calling model_fn.
Instructions for updating:
Please use `layer.__call__` method instead.
W0902 11:11:17.891770 140297466345280 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0902 11:11:18.716516 140297466345280 regularizers.py:99] 

Instructions for updating:
Use standard file utilities to get mtimes.
W0902 11:11:25.854440 140297466345280 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow_core/python/training/saver.py:1069: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
I0902 11:11:26.575459 140297466345280 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0902 11:11:26.773563 140297466345280 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2646 into training/model.ckpt.
I0902 11:11:31.884577 140297466345280 basic_session_run_hooks.py:606] Saving checkpoints for 2646 into training/model.ckpt.
2020-09-02 11:11:36.954945: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully

KeyboardInterrupt: 

In [4]:
# aquí nos vamos a fijar en los archivos model.ckpt y vamos a buscar el que tenga el número más alto, ya que este número
# indica el step del proceso de entrenamiento en el que se realizó el checkpoint del modelo
!dir training

checkpoint
ejemplo_labels_txt.PNG
eval_0
events.out.tfevents.1599036874.bigdata-alineware
faster_rcnn_inception_v2_coco.config
graph.pbtxt
label_map.pbtxt
labels.txt
mask_rcnn_inception_v2_coco.config
model.ckpt-0.data-00000-of-00001
model.ckpt-0.index
model.ckpt-0.meta
model.ckpt-2646.data-00000-of-00001
model.ckpt-2646.index
model.ckpt-2646.meta
tfrecord


In [7]:
# ahora vamos a exportar este modelo como un grafo de inferencia. Este grafo se podrá usar para generar predicciones luego.
# Debemos sustituir los parámetros de entrada ["pipeline_config_path","trained_checkpoint_prefix","output_directory"]
# por los nuestros. Es recomendable elegir un nombre reconocible para la carpeta de salida, como por ejemplo
# "inference_graph" seguido de la fecha y la hora
!python3 object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path training/mask_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix training/model.ckpt-2646 \
--output_directory inference_graph_02_09_10_14

Instructions for updating:
Please use `layer.__call__` method instead.
W0902 09:19:03.945902 139913162262336 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0902 09:19:04.757903 139913162262336 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I0902 09:19:04.847419 139913162262336 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0902 09:19:04.847673 139913162262336 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0902 09:19:04.880546 139

242 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz

2020-09-02 09:19:07.917132: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-02 09:19:08.038904: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:17:00.0
2020-09-02 09:19:08.039523: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:65:00.0
2020-09-02 09:19:08.098558: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-09-02 09:19:08.119623: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-09-02 09:19:08.127511: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0902 09:19:12.242301 139913162262336 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow_core/python/tools/freeze_graph.py:233: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
W0902 09:19:12.242529 139913162262336 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow_core/python/framework/graph_util_impl.py:277: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 360 variables.
I0902 09:19:12.670901